In [1]:
%%capture
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install nvidia-smi
!pip install mxnet-cu92

In [2]:
!wget https://www.openslr.org/resources/42/km_kh_male.zip
!unzip km_kh_male.zip
!ls km_kh_male

--2022-06-18 14:03:02--  https://www.openslr.org/resources/42/km_kh_male.zip
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/42/km_kh_male.zip [following]
--2022-06-18 14:03:03--  http://us.openslr.org/resources/42/km_kh_male.zip
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 866086951 (826M) [application/zip]
Saving to: ‘km_kh_male.zip’

km_kh_male.zip      100%[===================>] 825.96M  20.8MB/s    in 42s     

2022-06-18 14:03:45 (19.9 MB/s) - ‘km_kh_male.zip’ saved [866086951/866086951]

Archive:  km_kh_male.zip
   creating: km_kh_male/
  inflating: km_kh_male/line_index.tsv  
   creating: km_kh_male/wavs/
  inflating: km_kh_male/wavs/khm_1161_39452109

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset

colnames=['path','drop','text'] 
df  = pd.read_csv('/content/km_kh_male/line_index.tsv',sep='\t',header=None,names=colnames)
df['path'] = '/content/km_kh_male/wavs/'+df['path'] +'.wav'

train, test = train_test_split(df, test_size=0.1)

train.to_csv('/content/km_kh_male/line_index_train.csv')
test.to_csv('/content/km_kh_male/line_index_test.csv')
common_voice_train = load_dataset('csv', data_files='/content/km_kh_male/line_index_train.csv',split= 'train')
common_voice_test = load_dataset('csv', data_files='/content/km_kh_male/line_index_test.csv',split = 'train')
df.head()

Using custom data configuration default-d63918c6ec7fe4b3


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:
/usr/local/lib/python3.7/dist-packages/tqdm/std.py:1133: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  for obj in iterable:


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d63918c6ec7fe4b3/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


Using custom data configuration default-1002ccbe8a6d93c6


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1002ccbe8a6d93c6/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


,path,drop,text
0,/content/km_kh_male/wavs/khm_0308_0011865648.wav,NaN,ស្ពាន កំពង់ ចម្លង អ្នកលឿង នៅ ព្រៃវែង ជា ស្ពាន ...
1,/content/km_kh_male/wavs/khm_0308_0032157149.wav,NaN,ភ្លើង កំពុង ឆាប ឆេះ ផ្ទះ ប្រជា ពលរដ្ឋ នៅ សង្កា...
2,/content/km_kh_male/wavs/khm_0308_0038959268.wav,NaN,អ្នក សុំ ទាន ដេក ប្រកាច់ ម្នាក់ ឯង ក្បែរ ខ្លោង...
3,/content/km_kh_male/wavs/khm_0308_0054635313.wav,NaN,ស្ករ ត្នោត ដែល មាន គុណភាព ល្អ ផលិត នៅ ខេត្ត កំ...
4,/content/km_kh_male/wavs/khm_0308_0055735195.wav,NaN,ភ្នំបាខែង មាន កម្ពស់ តែ ចិត សិប ម៉ែត្រ សោះ


In [5]:
common_voice_train = common_voice_train.remove_columns(["drop","Unnamed: 0"])
common_voice_test = common_voice_test.remove_columns(["drop","Unnamed: 0"])

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))


def show_rand(dataset,num_examples=10):
  df = dataset.sample(n = num_examples, random_state = 2) 
  display(HTML(df.to_html()))

In [7]:
common_voice_train

Dataset({
    features: ['path', 'text'],
    num_rows: 2615
})

In [8]:
show_random_elements(common_voice_train.remove_columns(["path"]), num_examples=2)

,text
0,រុស្ស៊ី មាន ព្រំ ប្រទល់ ជាប់ ប្រទេស ដល់ ទៅ ដប់ បួន ឯ ណោះ
1,ដំណាំ ត្រសក់ របស់ អ្នក ខេត្ត ត្បូងឃ្មុំ ផ្តល់ ផល ខ្ពស់


In [9]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [10]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x7f3048180200> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [11]:
show_random_elements(common_voice_train.remove_columns(["path"]))

,text
0,ខ្ញុំ ចង់ ទៅ ឡូសអាន់ជើលែស នៅ ឆ្នាំ ក្រោយ
1,ស្រី តូច ចំណាន ចែក ធៀប រៀបការ នៅ កោះពេជ្រ
2,នំ អន្សម ជ្រូក របស់ អ្នកស្រី កែវ ច័ន្ទបូរណ៍ មាន រសជាតិ ឆ្ងាញ់ ពិសា
3,លោក ស៊ីន ច័ន្ទឆាយ៉ា ដែល ជា កូន ប្រុស តែ មួយ របស់ លោក ស៊ិន ស៊ីសាមុត ទទួល មរណភាព
4,ពពក វិល លើ ភ្នំ បូកគោ
5,ទិន្នន័យ ក្នុង ការ រៀបចំ ក្រុង ភ្នំពេញ
6,តារា ចម្រៀង ម៉ៅ ស៊ីណាត ប៉ះ ជាមួយ ពិធីករ ឌួង សុគា ធ្វើ ឱ្យ អ្នក ទស្សនា សើច ផ្អើល ពេញ កម្មវិធី សំណើច តាម ភូមិ
7,លីណូវ៉ូ បាន ប្រើ បច្ចេកវិទ្យា ចុង ក្រោយ ផលិត ទូរសព្ទ ដែល មាន ទំហំ តូច ជាង គេ
8,បញ្ជី ឈ្មោះ អ្នក បោះឆ្នោត នៅ កម្ពុជា
9,ភូមិ គំរូ មួយ នៅ ប្រទេស ឥណ្ឌា ដាំ ដើម ឈើ មួយ រយ ដប់ មួយ ដើម រាល់ ពេល មាន ក្មេង ស្រី ម្នាក់ កើត


In [12]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [13]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

In [14]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [15]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 41,
 'ក': 24,
 'ខ': 22,
 'គ': 55,
 'ឃ': 19,
 'ង': 52,
 'ច': 15,
 'ឆ': 60,
 'ជ': 28,
 'ឈ': 17,
 'ញ': 63,
 'ដ': 5,
 'ឋ': 7,
 'ឌ': 61,
 'ឍ': 18,
 'ណ': 1,
 'ត': 37,
 'ថ': 27,
 'ទ': 69,
 'ធ': 62,
 'ន': 10,
 'ប': 65,
 'ផ': 47,
 'ព': 32,
 'ភ': 58,
 'ម': 9,
 'យ': 40,
 'រ': 35,
 'ល': 43,
 'វ': 53,
 'ស': 42,
 'ហ': 68,
 'ឡ': 6,
 'អ': 48,
 'ឥ': 29,
 'ឧ': 3,
 'ឪ': 12,
 'ឫ': 33,
 'ឬ': 31,
 'ឭ': 8,
 'ឮ': 13,
 'ឯ': 0,
 'ឱ': 50,
 'ា': 39,
 'ិ': 66,
 'ី': 64,
 'ឹ': 59,
 'ឺ': 38,
 'ុ': 2,
 'ូ': 44,
 'ួ': 11,
 'ើ': 45,
 'ឿ': 25,
 'ៀ': 36,
 'េ': 26,
 'ែ': 20,
 'ៃ': 16,
 'ោ': 23,
 'ៅ': 54,
 'ំ': 14,
 'ះ': 51,
 'ៈ': 21,
 '៉': 67,
 '៊': 30,
 '់': 70,
 '៌': 46,
 '៍': 56,
 '៎': 57,
 '៏': 34,
 '័': 49,
 '្': 4}

In [16]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [17]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

73

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [20]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [21]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [22]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [23]:
processor.save_pretrained("/content/wav2vec2-large-xlsr-nepali/checkpoint-4000")

In [24]:
common_voice_train[0]

{'path': '/content/km_kh_male/wavs/khm_6753_1049864117.wav',
 'text': 'ខេមរៈ ស្រីពៅ បាន ទស្សនិកជន នៅ ក្រៅ ប្រទេស គាំទ្រ '}

In [25]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [26]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

In [27]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [28]:
common_voice_train = common_voice_train.map(resample, num_proc=4)
common_voice_test = common_voice_test.map(resample, num_proc=4)

In [29]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

ipd.Audio(data=np.asarray(common_voice_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [30]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(common_voice_train[rand_int]["speech"]).shape)
print("Sampling rate:", common_voice_train[rand_int]["sampling_rate"])

Target text: កិច្ច ប្រតិបត្តិការ ដែល មាន កិច្ច ធានា 
Input array shape: (45056,)
Sampling rate: 16000


In [31]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [32]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, num_proc=4, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


In [33]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [34]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [35]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [36]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [37]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [38]:
model.freeze_feature_extractor()

In [39]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/wav2vec2-large-xlsr-nepali",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
 
  save_steps=1000,
  eval_steps=1000,
  logging_steps=1000,
  learning_rate=5e-4,
  warmup_steps=410,
  save_total_limit=1,
)

In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [41]:
!nvidia-smi

Sat Jun 18 14:14:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    34W / 250W |   2133MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
1000,2.709900,0.368941,0.460443,42.065300,6.918000
2000,0.142200,0.402173,0.406646,42.627200,6.827000
3000,0.065900,0.415609,0.356408,42.929600,6.779000
4000,0.042800,0.419435,0.350870,43.309600,6.719000


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return

TrainOutput(global_step=4100, training_loss=0.7228981886840448, metrics={'train_runtime': 23865.5102, 'train_samples_per_second': 0.172, 'total_flos': 2.009769947596164e+19, 'epoch': 50.0, 'init_mem_cpu_alloc_delta': 333678, 'init_mem_gpu_alloc_delta': 1262054400, 'init_mem_cpu_peaked_delta': 19163, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 3475555, 'train_mem_gpu_alloc_delta': 3798702592, 'train_mem_cpu_peaked_delta': 139196796, 'train_mem_gpu_peaked_delta': 4167725568})

In [43]:
model = Wav2Vec2ForCTC.from_pretrained("gagan3012/wav2vec2-xlsr-khmer").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("gagan3012/wav2vec2-xlsr-khmer")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [44]:
input_dict = processor(common_voice_test["input_values"][0], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


In [70]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 291
})

In [71]:
common_voice_test_transcription = load_dataset('csv', data_files='/content/km_kh_male/line_index_test.csv',split = 'train')

Using custom data configuration default-1002ccbe8a6d93c6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-1002ccbe8a6d93c6/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)


In [72]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription["text"][0].lower())

Prediction:
ហ្សិប ជា ឈ្មោះ តំបន់ មួយ នៅ ប៉ែក អឺរ៉ុប

Reference:
ហ្សិប ជា ឈ្មោះ តំបន់ មួយ នៅ ប៉ែក អឺរ៉ុប


In [73]:
import os 
os.chdir("/content")

!wget https://www.openslr.org/resources/43/ne_np_female.zip
!unzip ne_np_female.zip
!ls ne_np_female

--2022-06-18 21:22:50--  https://www.openslr.org/resources/43/ne_np_female.zip
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://us.openslr.org/resources/43/ne_np_female.zip [following]
--2022-06-18 21:22:51--  http://us.openslr.org/resources/43/ne_np_female.zip
Resolving us.openslr.org (us.openslr.org)... 46.101.158.64
Connecting to us.openslr.org (us.openslr.org)|46.101.158.64|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 800375645 (763M) [application/zip]
Saving to: ‘ne_np_female.zip’

ne_np_female.zip    100%[===================>] 763.30M  8.73MB/s    in 61s     

2022-06-18 21:23:52 (12.6 MB/s) - ‘ne_np_female.zip’ saved [800375645/800375645]

Archive:  ne_np_female.zip
   creating: ne_np_female/
  inflating: ne_np_female/line_index.tsv  
   creating: ne_np_female/wavs/
  inflating: ne_np_female/wa

In [74]:
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset

colnames=['path','sentence'] 
df  = pd.read_csv('/content/ne_np_female/line_index.tsv',sep='\t',header=None,names = colnames)
df['path'] = '/content/ne_np_female/wavs/'+df['path'] +'.wav'

train, test = train_test_split(df, test_size=0.1)

train.to_csv('/content/ne_np_female/line_index_train.csv')
test.to_csv('/content/ne_np_female/line_index_test.csv')
#common_voice_train = load_dataset('csv', data_files='/content/ne_np_female/line_index_train.csv',split= 'train')
#common_voice_test = load_dataset('csv', data_files='/content/ne_np_female/line_index_test.csv',split = 'train')

In [89]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re
from sklearn.model_selection import train_test_split
import pandas as pd
from datasets import load_dataset

#!wget https://www.openslr.org/resources/43/ne_np_female.zip
#!unzip ne_np_female.zip
#!ls ne_np_female
!mkdir cer
!wget -O cer/cer.py https://huggingface.co/ctl/wav2vec2-large-xlsr-cantonese/raw/main/cer.py



test_dataset = load_dataset('csv', data_files='/content/km_kh_male/line_index_test.csv',split = 'train')
wer = load_metric("wer")
cer = load_metric("cer")


processor = Wav2Vec2Processor.from_pretrained("gagan3012/wav2vec2-xlsr-khmer")
model = Wav2Vec2ForCTC.from_pretrained("gagan3012/wav2vec2-xlsr-khmer") 
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'  
resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
	speech_array, sampling_rate = torchaudio.load(batch["path"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
	inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

	with torch.no_grad():
		logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

	pred_ids = torch.argmax(logits, dim=-1)
	batch["pred_strings"] = processor.batch_decode(pred_ids)
	return batch

cer = load_metric("cer")

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["text"])))
print("CER: {:2f}".format(100 * cer.compute(predictions=result["pred_strings"], references=result["text"])))

mkdir: cannot create directory ‘cer’: File exists
--2022-06-18 21:53:34--  https://huggingface.co/ctl/wav2vec2-large-xlsr-cantonese/raw/main/cer.py
Resolving huggingface.co (huggingface.co)... 3.208.54.125, 18.204.39.151, 34.197.58.156, ...
Connecting to huggingface.co (huggingface.co)|3.208.54.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3826 (3.7K) [text/plain]
Saving to: ‘cer/cer.py’

cer/cer.py          100%[===================>]   3.74K  --.-KB/s    in 0s      

2022-06-18 21:53:34 (883 MB/s) - ‘cer/cer.py’ saved [3826/3826]



Using custom data configuration default-1002ccbe8a6d93c6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-1002ccbe8a6d93c6/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1



WER: 4.905063


ValueError: ignored

In [90]:
cer = load_metric("cer")

In [91]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

test_dataset = load_dataset('csv', data_files='/content/km_kh_male/line_index_test.csv',split = 'train')

processor = Wav2Vec2Processor.from_pretrained("gagan3012/wav2vec2-xlsr-khmer")
model = Wav2Vec2ForCTC.from_pretrained("gagan3012/wav2vec2-xlsr-khmer") 

resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
	speech_array, sampling_rate = torchaudio.load(batch["path"])
	batch["speech"] = resampler(speech_array).squeeze().numpy()
	return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
	logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["text"][:2])


Using custom data configuration default-1002ccbe8a6d93c6
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-1002ccbe8a6d93c6/0.0.0/2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c)
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:704: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return (input_length - kernel_size) // stride + 1


Prediction: ['ហ្សិប ជា ឈ្មោះ តំបន់ មួយ នៅ ប៉ែក អឺរ៉ុប ោោ', 'កងទ័ព កូរ៉េ ខាង ជើង និង កងទ័ព ចិន បាន ចូល កាន់កាប់ ទីក្រុង សេអ៊ូល នៅ ពេល សង្គ្រាម លោក']
Reference: ['ហ្សិប ជា ឈ្មោះ តំបន់ មួយ នៅ ប៉ែក អឺរ៉ុប', 'កងទ័ព កូរ៉េ ខាង ជើង និង កងទ័ព ចិន បាន ចូល កាន់កាប់ ទីក្រុង សេអ៊ូល នៅ ពេល សង្គ្រាម លោក']
